## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/wayne/Desktop/Analysis_Project/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavieng,PG,-2.5744,150.7967,83.68,70,98,11.81,light rain
1,1,Nioro,GM,13.3500,-15.7500,78.80,74,100,9.53,overcast clouds
2,2,Bambous Virieux,MU,-20.3428,57.7575,70.21,79,49,18.50,light rain
3,3,Grindavik,IS,63.8424,-22.4338,47.89,100,100,17.27,light intensity drizzle
4,4,Grand-Lahou,CI,5.1367,-5.0242,77.02,77,55,7.52,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 0
What is the maximum temperature you would like for your trip? 100


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kavieng,PG,-2.5744,150.7967,83.68,70,98,11.81,light rain
1,1,Nioro,GM,13.3500,-15.7500,78.80,74,100,9.53,overcast clouds
2,2,Bambous Virieux,MU,-20.3428,57.7575,70.21,79,49,18.50,light rain
3,3,Grindavik,IS,63.8424,-22.4338,47.89,100,100,17.27,light intensity drizzle
4,4,Grand-Lahou,CI,5.1367,-5.0242,77.02,77,55,7.52,broken clouds
5,5,Jamestown,US,42.0970,-79.2353,65.01,67,97,1.01,overcast clouds
6,6,Greenwood,US,34.1668,-82.1165,81.28,78,0,0.00,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,73.31,68,36,13.65,scattered clouds
8,8,Esperance,AU,-33.8667,121.9000,58.84,69,87,30.69,overcast clouds
9,9,Butaritari,KI,3.0707,172.7902,82.27,67,0,14.56,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                4
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,83.68,light rain,-2.5744,150.7967,
1,Nioro,GM,78.80,overcast clouds,13.3500,-15.7500,
2,Bambous Virieux,MU,70.21,light rain,-20.3428,57.7575,
3,Grindavik,IS,47.89,light intensity drizzle,63.8424,-22.4338,
4,Grand-Lahou,CI,77.02,broken clouds,5.1367,-5.0242,
5,Jamestown,US,65.01,overcast clouds,42.0970,-79.2353,
6,Greenwood,US,81.28,clear sky,34.1668,-82.1165,
7,Rikitea,PF,73.31,scattered clouds,-23.1203,-134.9692,
8,Esperance,AU,58.84,overcast clouds,-33.8667,121.9000,
9,Butaritari,KI,82.27,clear sky,3.0707,172.7902,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not foun

In [9]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Kavieng,PG,83.68,light rain,-2.5744,150.7967,Nusa Island Retreat
2,Bambous Virieux,MU,70.21,light rain,-20.3428,57.7575,Casa Tia Villa
3,Grindavik,IS,47.89,light intensity drizzle,63.8424,-22.4338,Northern Light Inn & Max's Restaurant
4,Grand-Lahou,CI,77.02,broken clouds,5.1367,-5.0242,Hotel Beatitude de Grand Lahou
5,Jamestown,US,65.01,overcast clouds,42.0970,-79.2353,Hampton Inn & Suites Jamestown


In [10]:
# 8a. Create the output File (CSV)
output_data_file = 'vacation_search.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))